## API Academia Canaria de la Langua

Las API de la ACL se pueden consultar sin necesidad de autentificación. Todas las operaciones
son con `GET` y devuelven como resultado un JSON. El resultado _siempre_ es un diccionario. Este diccionario tendrá _siempre_ un campo `status`.

- Si todo ha ido bien, el campo `status` tendrá como contenido `ok`, y habrá
  otro campo en el diccionario con el nombre `result` que tendrá el resultado
  de la api, que podrá ser un número, diccionario, lista, o cualquier otro
  resultado válido en JSON.

- Si se ha producido un error, en la clave `status` encontraremos el valor `error`, __no__ habrá
  un campo `result`, pero si habrá un campo `message` con más información acerca del error
  peoducido.

Esto son los campos incluidos por defecto, pero dependiendo de la llamada, puede
que haya más claves con meta-informnación sobre el resultado, algunos
de los más habituales son:

- `timestamp`: marca temporal que indica el momento en que se realizó la respuesta. Como
  JSON no tiene un tipo especifico para fechas/timestamps, el valor es una
  cadena de texto en formato [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601). Si
  en el resultado hubieran también fechas o timestamp, se codificarían igual.

- `name`: Nombre de la API que se ha invocado.

- `length`: Si el dato devuelto en el campo `result` tienen el concepto 
  de longitud, tendría ese valor. Por ejemplo, si el resultado es una lista
  o un diccionario, en este campo vendrían el número de elementos. Si el
  resultado es una string de texto, la longitud de la misma. Si es un entero,
  no vendrá este campo, ya que no tiene sentido en ese caso.

### Versión de la API

La llamada API más sencilla de todas es la que nos permite comprobar la versión de la API activa.

Para consultarla, solo hay que hacer un GET a la ruta `/api/version/` en el website de la Academia, `wwww.academiacanarialengua.org`. El siguiente código realiza una consulta a esa API usando la librería [requests](http://docs.python-requests.org/en/master/) de Python.

In [1]:
import requests
host = 'www.academiacanarialengua.org'
url = 'http://{host}/api/version/'.format(host=host)

data = requests.get(url)
print(data.json())

{'status': 'ok', 'result': '0.1', 'timestamp': '2018-06-29 22:19:33.400743', 'length': 3, 'name': 'version'}


Ejemplo de consulta usando el navegador:

- [http://www.academiacanarialengua.org/api/version/](http://www.academiacanarialengua.org/api/version/)

### la API de búsqueda

La api de búsqueda está en la ruta `api/search/<raiz>`, de forma que podemos buscar todas las palabras que _empiezan_
por una serie de letras, por ejemplo, para buscar las palabras que empiezan por `agu`, la URL a usar
sería [http://www.academiacanarialengua.org/api/search/agu](http://www.academiacanarialengua.org/api/search/agu). El siguiente ejemplo lista las palabras encontradas usando esa misma llamada:

In [2]:
import requests
host = 'www.academiacanarialengua.org'
term = 'agu'
url = 'http://{host}/api/search/{term}'.format(host=host, term=term)

r = requests.get(url)
data = r.json()
print(data['status'])
for row in data['result']:
    print(row['id_entrada'], row['palabra'])

ok
525 agua
526 aguacatero
527 aguachento
528 aguachirre
529 aguadija
530 aguadura
531 aguafeble
532 aguaidil
533 aguaje
534 aguantamicuna
535 aguantar
536 aguapata
537 aguasperarse
538 aguaviva
539 agüero
540 aguiciar
541 águila
542 aguililla
543 agüimense
544 agujerada
545 agujeta
546 agujilla
547 agulense


### La API de consulta de detalles de una entrada

A partir del identificador de cada entrada, que es el número `id_entrada` devuelto por la API
de búsqueda, podemos obrtener todos los datos de esa entrada, especialmente la lista de acepciones
del término. La api esta localizada en `api/word/<id. entrada>`. 

Por ejemplo, podemos obtener los detalles de la entrada _aguaviva_ (Id. 538) con la siguiente
llamada:

- [http://www.academiacanarialengua.org/api/word/538](http://www.academiacanarialengua.org/api/word/538)

In [3]:
from IPython.core.display import display, HTML
import requests

host = 'www.academiacanarialengua.org'
id_word = 538
url = 'http://{host}/api/word/{pk}'.format(host=host, pk=id_word)

data = requests.get(url).json()
word = data['result']
for k in word:
    if k != 'acepciones':  # Veremos las acepciones más adelante
        print(k, data['result'][k])

id_entrada 538
version 0
destacada True
palabra aguaviva
slug aguaviva
drae False
variante 
revisada True
edicion 1


#### Descripción de los campos

Los campos de una palabra son los siguientes:

- **`id_entrada`**: La clave primaria e identificador único de cada entrada en el 
  diccionario.

- **`edicion`**: Se refiere a la edición en papel. Estos contenidos son los que están publicados
    en la versión actual, que es la 1. Hay una versión en desarrollo en la que todas
    las entradas tendrán en este valor un 2, pero por el momento no es pública
    
- **`version`**: La versión de la entrada. Hay alguna entradas que corresponden 
  a palabras que se escriben igual pero resultan ser entradas diferentes. Véase por 
  ejemplo [baifa](http://www.academiacanarialengua.org/palabra/baifa/), que aparece 
  dos veces, con id. 5112, version 1 e id. 860, version 2. A pesar de que se escriben
  igual, son dos entradas distintas en el diccionario, porque
  la versión 1 es un sustantivo y la 2 es un adjetivo. Si `version` es cero significa que
  no hay ninguna otra entrada que se escriba igual que esta. Si hay duplicados, `version`
  valdra $1$, $2$,...

- **`palabra`**: El texto de la palabra, siempre en minúscula.

- **`slug`**: Se usa para componer el URL de la palabra en la web. Normalemente coincide
  con la palabra, pero a veces es una versión más sencilla, sin acentos, eñes, etc...
  para que las direcciones se puedan escribir usando cualquier teclado. Por ejemplo
  [abejón](http://www.academiacanarialengua.org/palabra/abejon) tiene como 
  slug `abejon`. Observese que el campo `slug` es la última parte de la ruta. 
  
  La combinación de `slug` y `version` forman
  un par de valores único para cada palabra en el diccionario y son, por tanto, una 
  clave candidata u opcional.

- **`variante`**: Se usa para palabras que tienen una variacíón por género o por otras
  causas, véase por 
  ejemplo [abaifado/da](http://www.academiacanarialengua.org/palabra/abaifado). El valor
  del campo `variante` en este caso es `da`.
  
- **`revisada`**: Si está a verdadero, es que la entrada está revisada. Todas las entradas
  en el diccionario actual están revisadas, es decir, todas tienen este campo a `True`.
  
- **`drae`**: Indicaría si esta palara está recogida en DRAE o no. No es un dato fiable.
  
- **`destacada`**: Este campo es un booleano que indica si la palabra debe aparecer un la
  pagina principal de la web de la ACL. Asi evitamos que aparezcan palabras con poco
  contenido, sinónimos, etc...
  


### Las acepciones

Además de lo anterior, la palabra tiene una lista de acepciones. Cada una de las
acepciones tiene los siguientes campos.

- **`id_acepcion`**: El identificador único y clave primaria de la acepción

- **`orden`**: El orden en que se debe presentar cada acepción

- **`formato`**: Siempre debbería ser `html`

- **`texto`**: El lexto de la acepción.

- **`etiquetas`**: Una lista de las etiquetas asociadas a la acepción. Cada
  etiqueta se compone de 5 campos:
  
  - **`id_etiqueta`**: El identificador de la etiqeuta
  
  - **`descripcion`**: La descripción textual
  
  - **`etiqueta`**: La marca usada para indicar esa etiqueta, p.e. `m` para
    sustantivo masculino.
    
  - **`orden`**: El orden en que se deben mostrar las etiquetas.  
    
  - **`grupo`**: Una letra que indica el grupo al que corresponde la etiqueta. Hay tres grupos
    definidos en este diccionario: Gramatical (`G`), Diatópicas (`L`) y Otras (`X`).

- **`referencias`**: ⭐⭐ En el caso de que el texto de la acepción haga referencia a palabras
    contenidas en el diccionario, se indicará en este campo. Es una lista de referencias,
    cada una de las cuales es un diccionario que contiene los siguentes valores.
    
    - **`id_entrada`**: El identificador numérico de la entrada a la que se referencia
    
    - **`palabra`**: La palabra de dicha entrada
    
    - **`texto_enlace`**: La cedena de texto que, incluida en el texto de la acepción,
      debe usarse como texto resaltado o vínculo. En la mayoría de los casos
      se corresponde con el texto en `palabra`, pero hay casos en que no, por ejemplo
      si se usa el plural. Véase, por ejemplo, 
      [**palmero**](http://www.academiacanarialengua.org/palabra/palmero/)
      ([3553](http://www.academiacanarialengua.org/api/word/3553/))
      ,
      que en su cuarta acepción hace referencia a la entrada
      [**penca**](http://www.academiacanarialengua.org/palabra/penca/)
      ([3692](http://www.academiacanarialengua.org/api/word/3692/)) pero
      usando como texto de enlace la palabra *pencas*.


Veamos un ejemplo donde mostramos las acepciones de la palabra _aguaviva_ (Id. 538):

In [4]:
from IPython.core.display import display, HTML
import requests

host = 'www.academiacanarialengua.org'
id_word = 538
url = 'http://{host}/api/word/{pk}'.format(host=host, pk=id_word)

data = requests.get(url).json()
word = data['result']
display(HTML('<h4>Acepciones</h4>'))
display(HTML('<ul>'))
for acepcion in word['acepciones']:
    etiquetas = ','.join([
        etiqueta['etiqueta']
        for etiqueta in acepcion['etiquetas']
        ])
    display(HTML('<li><b>{}</b> <tt>{}</tt> {}</li>'.format(
        acepcion['orden'],
        etiquetas,
        acepcion['texto'],
    )))
    
display(HTML('</ul>'))

#### Acepciones como ejemplos ⭐

Algunas acepciones son en realidad ejemplos de uso. Se pueden distinguir porque
el texto empieza siempre con la cadena de texto `//`. Véase como ejemplo
la entrada **Aceite (id. 487)**.

 - [Aceite (web)](http://www.academiacanarialengua.org/palabra/aceite/)
 - [Aceite (api)](http://www.academiacanarialengua.org/api/word/487)

#### Acepciones que citan otras entradas ⭐

Algunas acepciones usan o _citan_ en su explicación otra entrada diferente del 
diccionario. En ese caso, en la acepción se incluye un campo
`cita` que apunta a la entrada usada. Por ejemplo, la entrada **abochacharse (id. 461)**, 
en su primera acepción, referencia a la entrada **hincharse (id. 2763)**.
El campo cita incluye los valores:

 - **`palabra`**. La palabra usada
 - **`id_entrada`**: El identificador numérico de la entrada
 
Vease como ejemplo:

 - [abochacharse] (web)](http://www.academiacanarialengua.org/palabra/abochacharse/)
 - [abochacharse (api)](http://www.academiacanarialengua.org/api/word/461)
 
 **Nota**: En la versión actual de la web, no se usa esta información, pero la aplicación
 móvil podría usarla, si lo consideráis pertinente.

#### Acepciones con notas ⭐

Algunas acepciones incluyen notas, ya sea sobre pronunciación o haciendo
referencia a alguna foto que está asociada a otra entrada. en esos casos,
la acepcion incluirá un campo `nota` adicional. Véase como ejemplo
la entrada **caracolear (id. 1322)**, que tiene una nota sobre pronunciación:

 - [caracolear (web)](http://www.academiacanarialengua.org/palabra/caracolear/)
 - [caracolear (api)](http://www.academiacanarialengua.org/api/word/1322)
 
O la entrada **tabaquero (id. 4508)**, que incluye una nota haciendo referencia
a la foto de otra entrada, **camello (id. 1254)**. En este caso, los campos `foto` y `thumbnail` estarán incluidos con la información de dicha foto (Véase la
sección: _Entradas con fotos_).

 - [tabaquero (web)](http://www.academiacanarialengua.org/palabra/tabaquero/)
 - [tabaquero (aip)](http://www.academiacanarialengua.org/api/word/4508)


#### La entrada que más acepciones tiene (Papa)⭐

He pensado que podría seros interesante saber que entrada tiene ahora
mismo más acepciones, para probar la interfaz, tiempos de respuesta, etc...
Se trata de **papa (id. 3582)**, con $23$ acepciones (y foto):

 - [papa (web)](http://www.academiacanarialengua.org/palabra/papa/)
 - [papa (api)](http://www.academiacanarialengua.org/api/word/3582)



### Entradas con fotos ⭐

las entradas que tengan foto incluirán doc campos más, `foto` y `thumbnail`. el campo `foto` incluye la siguiente información:
    
- **`url`**: La URl relativa de la foto.
- **`peso`**: El tamaño en bytes del fichero
- **`ancho`**: Ancho en pixels de la imagen
- **`alto`**: Alto en pixels de la imagen
- **`leyenda`**: Leyenda para incluir al pie de la foto

El campo `thumbnail` es una versión reducida en tamaño de la misma foto. Está
garantizado que su tamaño siempre será, como máximo, $320x200$ o $200x320$ (según
el formato sea apaisado o retrato). Los campos son los mismo que en la foto, pero
se omite la leyenda.

Vease como siguiente ejemplo la entrada **Abade (Id. 440)**:

 - [abade (web)](http://www.academiacanarialengua.org/palabra/abade/)
 - [abade (api)](http://www.academiacanarialengua.org/api/word/440)

#### Api de busqueda que se podrían hacer facilmente

No están hechas, pero estas API se podrían hacer muy facilmente:
    
- Búsqueda de palabras que **contengan** un determinado fragmento (no solo las que _empiezan por_)

- Búsqueda de palabras que incluya el texto de las acepciones.


### Palabras homógrafas ⭐⭐

Se llaman palabras **homónimas** a aquellas cuya pronunciación es igual o similar pero difieren en su significado. El DRAE da como ejemplo de palabras homónimas _aya_, ‘niñera’ y _haya_, ‘árbol’. Dentro de éstas, se distingue entre las palabras **homófonas** y las **homógrafas**. Las primeras se pronuncian igual pero se escriben de un modo diferente y tienen significados distintos, y las segundas difieren también en su significado pero la grafía es idéntica.

Obviamente, para nosotros el problema son las homógrafas. En la versión actual
del diccionario hay 53 palabras homógrafas, que se muestran en el siguiente listado. El enlace sobre el texto lleva a la vista de la palabra en la web, el enlace del id. numérico lleva a la API. En la web, se presentan las dos versiones de la palabra, una detras de la otra, y se disinguen con un superíndice:

In [7]:
%%HTML
<style>
th {
    text-align: left;
    weight: bolder;
    background-color: #E9E9E9;
    color: black;
}
</style>
<table>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/andoriña/">andoriña</a> (<a href="http://www.academiacanarialengua.org/api/word/679/">679</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/andoriña/">andoriña</a> (<a href="http://www.academiacanarialengua.org/api/word/5094/">5094</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> (<i>Apus unicolor</i>, <i>Apus pallidus brehmorum</i>, <i>Apus apus</i>) Nombre con el que se designa indistintamente a las tres especies de vencejos que nidifican en Canarias: vencejo unicolor, vencejo pálido y vencejo común. Son aves insectívoras que se alimentan en pleno vuelo y emiten unos característicos chillidos. Parcialmente migratorias, estas aves de unos quince centímetros son de color pardo negruzco. <i>Todavía me acuerdo de que a mi pueblo venían las andoriñas en bandadas volando bajito y siempre cazando</i>. En algunas zonas de Canarias se conocen también con los nombres de aburrión y golondrina.</>
<p><b>2</b> Persona inquieta y vivaz. <i>¿Y quién de nosotros es capaz de seguir el ritmo de esa andoriña</i><i>?</i></>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Dicho de una cabra, totalmente negra. <i>Hay pastores que dicen que las cabras moriscas son más lecheras que las andoriñas</i>.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/bago/">bago</a> (<a href="http://www.academiacanarialengua.org/api/word/858/">858</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/bago/">bago</a> (<a href="http://www.academiacanarialengua.org/api/word/5129/">5129</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Cada uno de los frutos que con otros iguales forman un racimo, como el de la uva o el plátano. <i>Cogió un baguito de uva y se lo echó a la boca.</i></>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Dicho de ciertos frutos, especialmente de la aceituna, que tienen muy poca pulpa porque no alcanzan su desarrollo normal. U.?t. c. s. <i>Mi madre, antes de ponerlas en su mojo, separaba las aceitunas bagas de las buenas</i>.</>
<p><b>2</b> Fruto de ciertas plantas, como el barbusano, el laurel o la pitera, que contiene una sola semilla, relativamente grande, recubierta de una capa carnosa delgada. <i>De la baga de loro se llegó a hacer aceite</i>.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/baifa/">baifa</a> (<a href="http://www.academiacanarialengua.org/api/word/860/">860</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/baifa/">baifa</a> (<a href="http://www.academiacanarialengua.org/api/word/5112/">5112</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> <i>Baifo</i> hembra. <i>Vendió una baifita en treinta euros</i>.</>
<p><b>2</b> Cabra de menos de un año, que puede ser cubierta.</>
<p><b>3</b> Modorra, soñolencia. <i>Después de comer, me entra una baifa, que no me deja hacer nada</i>.</>
<p><b>4</b> // <b>estar (loco) como una baifa</b>. Estar loco, chiflado.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Se dice de la papa fallida.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/barbusano/">barbusano</a> (<a href="http://www.academiacanarialengua.org/api/word/905/">905</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/barbusano/">barbusano</a> (<a href="http://www.academiacanarialengua.org/api/word/5096/">5096</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> (<i>Apollonias barbujana</i>) Árbol de la familia de las lauráceas, propio de las Islas Canarias y Madeira. Es de alto porte y su madera, extremadamente dura y resistente, es algo parecida a la caoba. <i>Los árboles más altos eran los tiles, los mocanes y los barbusanos</i>.</>
<p><b>2</b> Madera de este árbol. <i>Los ebanistas llaman al barbusano la caoba canaria</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Dicho de una cabra, blanca con el hocico rayado de negro.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/belete/">belete</a> (<a href="http://www.academiacanarialengua.org/api/word/957/">957</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/beleté/">beleté</a> (<a href="http://www.academiacanarialengua.org/api/word/5144/">5144</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> <i>Beletén</i>, primera leche que da la hembra después de parida.</>
<p><b>2</b> Leche que dan las hembras los dos primeros días después del parto.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> <i>Beletén</i>, primera leche que da la hembra después de parida.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/bicácaro/">bicácaro</a> (<a href="http://www.academiacanarialengua.org/api/word/987/">987</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/bicácaro/">bicácaro</a> (<a href="http://www.academiacanarialengua.org/api/word/5097/">5097</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Fruto de la <i>bicacarera</i>. <i>Los guanches se comían los bicácaros</i>.</>
<p><b>2</b> <i>Bicacarera</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Se dice del ojo y de la mirada torcida. <i>Este niño tiene un ojo medio bicácaro</i>.</>
<p><b>2</b> Que tiene el ojo o la mirada torcida.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/borrallo/">borrallo</a> (<a href="http://www.academiacanarialengua.org/api/word/5113/">5113</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/bote/">bote</a> (<a href="http://www.academiacanarialengua.org/api/word/5098/">5098</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Dicho de una cabra, de color ceniciento.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Embarcación sin cubierta, más pequeña que el barco de pozo, para pescar en el litoral. <i>En un descuido vino una ola y les reviró el bote</i>.</>
<p><b>2</b> Pequeña embarcación auxiliar con proa en ángulo y popa chata. <i>Salieron a pescar en la falúa, y dejaron el bote fondeado</i>.</>
<p><b>3</b> Embarcación que se utiliza en el deporte de la vela latina, que puede alcanzar hasta los seis metros y medio de eslora. <i>Todos los domingos se iba a la avenida a ver la regata de botes</i>.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/bote/">bote</a> (<a href="http://www.academiacanarialengua.org/api/word/1086/">1086</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/cañizo/">cañizo</a> (<a href="http://www.academiacanarialengua.org/api/word/5117/">5117</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> (<i>Delphinus delphis</i>) Especie de delfín, mamífero cetáceo.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Dicho de una persona, delgada. <i>A pesar de todo lo que comía, era muy cañizo</i>.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/carajaca/">carajaca</a> (<a href="http://www.academiacanarialengua.org/api/word/5106/">5106</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/carajaca/">carajaca</a> (<a href="http://www.academiacanarialengua.org/api/word/5107/">5107</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Plato compuesto por hígado de res vacuna cortado fino y troceado, frito y aderezado con salsa algo picante. <i>La especialidad de este bar son las carajacas y las vueltas de carne</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> (<i>Maja brachydactyla</i>) Crustáceo marino, variedad grande de cangrejo, que posee una concha muy dura y escabrosa con muchas púas y cinco pares de patas muy largas. <i>Aquella bahía tenía un fondo buenísimo: en una mañana se podía coger hasta una docena de carajacas</i>. En otras zonas de Canarias se conoce con el nombre de <i>santorra</i>.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/carisco/">carisco</a> (<a href="http://www.academiacanarialengua.org/api/word/1332/">1332</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/cárisco/">cárisco</a> (<a href="http://www.academiacanarialengua.org/api/word/5136/">5136</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> (<i>Persea indica</i>) Árbol de la familia de las lauráceas, de hasta veinte metros de altura, con hojas lanceoladas de color verde pálido, que se vuelven rojizas al envejecer. Su fruto, de unos dos centímetros, tiene forma de aceituna y, al madurar, es de color negro azulado. Es frecuente en los bosques de <i>laurisilva</i>. En otras zonas de Canarias se conoce con los nombres de <i>viñático</i> y <i>viñátigo</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Carisco.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/chopa/">chopa</a> (<a href="http://www.academiacanarialengua.org/api/word/1536/">1536</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/chopa/">chopa</a> (<a href="http://www.academiacanarialengua.org/api/word/1537/">1537</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Cucaracha, cuando es de tamaño grande.</>
<p><b>2</b> (<i>Rhyparobia maderae</i>) Cucaracha grande de color crema oscuro, que suele emprender vuelo en días de calor. En otras zonas de Canarias se conoce con los nombres de <i>cuca volona</i> y <i>salema</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Nariz, especialmente la abultada. <i>Con el catarro, tenía la chopa roja de tanto sonarse</i>.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/chucho/">chucho</a> (<a href="http://www.academiacanarialengua.org/api/word/1554/">1554</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/chucho/">chucho</a> (<a href="http://www.academiacanarialengua.org/api/word/1555/">1555</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Látigo.</>
<p><b>2</b> Castigo físico. Se emplea sobre todo en expresiones como <i>dar chucho</i>, <i>alcanzar chucho</i>, etc. <i>Siempre lo amenazaba con darle chucho, pero nunca le llegó a pegar</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Nombre que se da a varias especies de peces con cuerpo en forma de raya, de cabeza puntiaguda o redondeada, cuerpo aplastado, liso y de color pardo o negruzco. Algunas de estas especies pueden alcanzar hasta los dos metros y medio de longitud y más de doscientos kilos de peso. Tienen una cola larga, que está provista de una aguda espina venenosa. Viven en aguas costeras, sobre fondos de arena y lodo, desde la orilla hasta los doscientos metros de profundidad. Aunque, por lo general, no suele consumirse fresca, su carne se aprovecha a veces para hacer <i>tollos</i>. <i>Los latigazos que daba el chucho con el rabo eran terribles</i>. En algunas zonas de Canarias se conocen también con el nombre de <i>chucha</i>.</>
<p><b>2</b> // <b>ser más feo que un chucho</b>. Ser muy poco agraciado.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/corujo/">corujo</a> (<a href="http://www.academiacanarialengua.org/api/word/1690/">1690</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/corujo/">corujo</a> (<a href="http://www.academiacanarialengua.org/api/word/5151/">5151</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Fantasma con cuya mención se pretende amedrentar a los niños. <i>Nos decían que, si no nos portábamos bien, venía el corujo y nos llevaba</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Dicho de una cabra, de pelaje en que se combinan manchas negras y <i>canelas</i>.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/cuca/">cuca</a> (<a href="http://www.academiacanarialengua.org/api/word/1731/">1731</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/cuca/">cuca</a> (<a href="http://www.academiacanarialengua.org/api/word/1732/">1732</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Cucaracha. <i>Este verano, como hizo mucho calor, se le llenó la casa de cucas</i>.</>
<p><b>2</b> Órgano sexual masculino, especialmente el de los niños.</>
<p><b>3</b> // <b>cuca inglesa</b>. (<i>Blatella germanica</i>) Cucaracha pequeña de color rubio. Se encuentra principalmente en las zonas costeras de clima cálido. En otras zonas de Canarias se conoce con el nombre de <i>chiripa</i>.</>
<p><b>4</b> // <b>cuca volona</b>. (<i>Rhyparobia maderae</i>) Cucaracha grande de color crema oscuro, que suele emprender vuelo en días de calor. En algunas zonas de Canarias se conoce también con los nombres de <i>chopa</i> y <i>salema</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Aplicado a cabras, de color negro.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/empanado/">empanado</a> (<a href="http://www.academiacanarialengua.org/api/word/5145/">5145</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/empanado/">empanado</a> (<a href="http://www.academiacanarialengua.org/api/word/1960/">1960</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Conjunto de panas de una embarcación, dispuestas de proa a popa. <i>Hay que sacar todo el empanado para poder limpiar bien el barco por dentro</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Dicho de una plantación, que tiene sus elementos muy juntos o apretados.<i> Esta cebada creció bien empanadita.</i></>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/fato/">fato</a> (<a href="http://www.academiacanarialengua.org/api/word/5131/">5131</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/fato/">fato</a> (<a href="http://www.academiacanarialengua.org/api/word/5132/">5132</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Vaho, mal aliento.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Trapo, andrajo. <i>Se conoce que iba a pulpear, porque llevaba un fato y una fija en la mano.</i></>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/funche/">funche</a> (<a href="http://www.academiacanarialengua.org/api/word/2443/">2443</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/funche/">funche</a> (<a href="http://www.academiacanarialengua.org/api/word/2444/">2444</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Pequeño departamento dentro de las viviendas, comúnmente de reducidas dimensiones y desprovisto de luz y ventilación, que sirve de trastero.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Millo sin tostar molido muy fino.</>
<p><b>2</b> Vianda que se prepara con este millo molido. Por lo general, se come en frío añadiéndole leche o tabefe. Modernamente se toma también como postre, agregándole otros ingredientes, como canela en rama, corteza de limón, pasas y almendras.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/galletón/">galletón</a> (<a href="http://www.academiacanarialengua.org/api/word/2476/">2476</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/galletón/">galletón</a> (<a href="http://www.academiacanarialengua.org/api/word/2477/">2477</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Adolescente, muchacho. <i>Aunque era un galletón, todavía iba a buscar nidos de pájaros</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Galleta grande. <i>Para desayunar le daban un galletón y un vaso de leche</i>.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/gavia/">gavia</a> (<a href="http://www.academiacanarialengua.org/api/word/2548/">2548</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/gavia/">gavia</a> (<a href="http://www.academiacanarialengua.org/api/word/2549/">2549</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Parcela de terreno para cultivar, bordeada por un caballón de tierra, que se riega por encharcamiento con el agua de lluvia procedente de la alcogida. <i>Todo eso eran gavias, y ahora está todo lleno de casas</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Parte más alta de un árbol.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/guanil/">guanil</a> (<a href="http://www.academiacanarialengua.org/api/word/2651/">2651</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/guanil/">guanil</a> (<a href="http://www.academiacanarialengua.org/api/word/5146/">5146</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Dicho del ganado que se cría suelto, que no tiene marca que identifique al dueño. <i>En la apañada se marcaban las crías que estaban guaniles</i>.</>
<p><b>2</b> Dicho de persona, animal o cosa, que, teniendo pareja, se queda solo. <i>Ya no le queda más que una tarraya vieja y un trasmallo guanil</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Se dice de la persona de corta edad.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/majada/">majada</a> (<a href="http://www.academiacanarialengua.org/api/word/3132/">3132</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/majada/">majada</a> (<a href="http://www.academiacanarialengua.org/api/word/3133/">3133</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Cerca móvil y desmontable formada por lances, que se instala en el campo y que se usa para encerrar el rebaño de ovejas.</>
<p><b>2</b> Depresión que se forma en la ladera de una montaña.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Acción y efecto de majarse, machacarse o aplastarse algún miembro del cuerpo.</>
<p><b>2</b> Acción y efecto de majarse, aprisionarse con daño la piel o algún miembro del cuerpo.</>
<p><b>3</b> Serie de golpes dados a una persona que la dejan maltrecha, paliza.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/mano/">mano</a> (<a href="http://www.academiacanarialengua.org/api/word/3203/">3203</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/mano/">mano</a> (<a href="http://www.academiacanarialengua.org/api/word/3204/">3204</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Fórmula apelativa, aféresis de <i>hermano</i>, usada con el valor de compañero, que se emplea entre iguales y en situaciones de habla muy informales. <i>Mano, termina uno cansándose de tanto abuso</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Manilla.</>
<p><b>2</b> Cada uno de los racimos de fruto que echa el tomatero.</>
<p><b>3</b> Grupo de personas, comúnmente de baja calidad. <i>En esta plaza se junta todas las noches una mano de golfantes y gamberros</i>.</>
<p><b>4</b> En expresiones como <i>una mano de estampidos</i> o <i>una mano de estacazos</i>, gran cantidad o gran número. <i>Tanto hizo, hasta que alcanzó una mano de cogotazos</i>.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/margullo/">margullo</a> (<a href="http://www.academiacanarialengua.org/api/word/3240/">3240</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/margullo/">margullo</a> (<a href="http://www.academiacanarialengua.org/api/word/3241/">3241</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Acción de margullar o margullir.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Marullo.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/mimo/">mimo</a> (<a href="http://www.academiacanarialengua.org/api/word/3346/">3346</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/mimo/">mimo</a> (<a href="http://www.academiacanarialengua.org/api/word/3347/">3347</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> (<i>Nicotiana glauca</i>) Arbusto de la familia de las solanáceas, con hojas lanceoladas y flores amarillas de forma tubular, muy abundante en lugares donde se vierten escombros y en los bordes de la carretera. En otras zonas de Canarias se conoce con los nombres de <i>bobo</i> y <i>tabaco moro</i>.</>
<p><b>2</b> (<i>Fuchsia magellanica</i>) Pequeña planta arbustiva de jardín, de la familia de las oenoteráceas, con flores de color rojizo en forma de pequeña campanita.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Dulce de forma redondeada y cocido al horno, hecho con clara de huevo, azúcar y raspas de limón. <i>En unos sitios le dicen mimo y en otros, merengue</i>.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/mole/">mole</a> (<a href="http://www.academiacanarialengua.org/api/word/3370/">3370</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/mole/">mole</a> (<a href="http://www.academiacanarialengua.org/api/word/3371/">3371</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Mol.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Blando. <i>Los plátanos, si se pasan de maduros, se ponen moles</i>.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/mujo/">mujo</a> (<a href="http://www.academiacanarialengua.org/api/word/3443/">3443</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/mujo/">mujo</a> (<a href="http://www.academiacanarialengua.org/api/word/5149/">5149</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Musgo, planta.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Musgo.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/musgo/">musgo</a> (<a href="http://www.academiacanarialengua.org/api/word/5150/">5150</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/musgo/">musgo</a> (<a href="http://www.academiacanarialengua.org/api/word/3455/">3455</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Nombre que se da, además de a las plantas muscíneas, a las algas y a los líquenes.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Se dice de la cabra que tiene las orejas más pequeñas que las de tamaño normal.</>
<p><b>2</b> Se dice de la cabra que tiene las orejas más grandes que la muesa y más pequeñas que las de tamaño normal.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/ñoño/">ñoño</a> (<a href="http://www.academiacanarialengua.org/api/word/5086/">5086</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/ñoño/">ñoño</a> (<a href="http://www.academiacanarialengua.org/api/word/5124/">5124</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Dedo del pie. U. m. en pl.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Se dice de la persona que tiene la mandíbula inferior más saliente que la superior. <i>Yo soy ñoña, pero mi hija no salió a mí en eso</i>.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/papa/">papa</a> (<a href="http://www.academiacanarialengua.org/api/word/3582/">3582</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/papá/">papá</a> (<a href="http://www.academiacanarialengua.org/api/word/5123/">5123</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> (<i>Solanum tuberosum</i>) Planta de la familia de las solanáceas, originaria de América. <i>El cultivo de la papa se introdujo en Canarias en el siglo XVI</i>.</>
<p><b>2</b> Tubérculo de dicha planta. <i>En la comida del canario no podían faltar las papas y el gofio</i>. (Fig. 109)</>
<p><b>3</b> Inflamación de las glándulas parótidas. <i>Tenía unas papas tremendas en la garganta, y no podía ni tragar</i>.</>
<p><b>4</b> Abultamiento cutáneo. <i>Le salió una papa en el pescuezo</i>.</>
<p><b>5</b> Tanto. <i>Vaya papa metió el delantero centro</i>.</>
<p><b>6</b> // <b>papa arrugada</b>. La que es guisada con cáscara y bastante sal.</>
<p><b>7</b> // <b>papa autodate</b>. Variedad de <i>papa</i>, alargada y de color blanco, muy estimada para comer. <i>Ya tú ves: la papa autodate es buenísima para comer, pero muy pudrilona</i>.</>
<p><b>8</b> // <b>papa azucena</b>. Clase de <i>papa de color</i>, de pulpa blanquecina, la cual presenta dos variedades distintas, que, de acuerdo con el color de la cáscara, se denominan <i>azucena blanca</i> y <i>azucena negra</i>. <i>Para acompañar el pescado, nos arrugaron unas papas azucenas, que estaban buenísimas</i>.</>
<p><b>9</b> // <b>papa blanca</b>. <i>Papa</i> de semilla foránea, importada generalmente de las Islas Británicas, la cual es de tamaño mediano o grande, de pulpa algo aguanosa, con pocos ojos, y cáscara de diversos colores, aunque predominan las gamas blanca, amarillosa y rosácea. Se emplea sobre todo para freír y para caldos y potajes, pero también para guisar, tanto pelada como con cáscara. <i>Hoy la papa blanca que más se está vendiendo es la quinegua, la rosada y la autodate</i>. <b>2</b>.Variedad de <i>papa</i>, redonda y de color blanco, con muchos ojos. Es una clase poco cultivada en la actualidad, debido a que es dura para comer.</>
<p><b>10</b> // <b>papa bonita</b>. Variedad de <i>papa de color</i>, de cáscara blanca amarillosa y pulpa compacta y azucarada. Es la más abundante y conocida de dicha clase de <i>papas</i>. <i>Fui a la recova a comprar papas bonitas, pero están por las nubes</i>. <b>2</b> <i>Tf.</i> <i>Papa de color</i>.</>
<p><b>11</b> // <b>papa borralla</b>. Variedad de <i>papa de 
color</i>, algo mayor que la <i>bonita</i> y de superficie más irregular, debido a 
que presenta muchos ojos y más hondos que los de esta. Su pulpa es algo más seca y compacta. <i>Las papas borrallas de la zona de Anaga dicen que son las mejores</i>.</>
<p><b>12</b> // <b>papa cara</b>. Variedad híbrida de <i>papa</i> de importación, que, aunque se considera que no es de la mejor calidad, se cultiva por su gran rendimiento.</>
<p><b>13</b> // <b>papa chinegua</b>. <i>Papa quinegua</i>.</>
<p><b>14</b> // <b>papa cría</b>. (<i>Terfezia claveryi</i>) Especie de trufa de forma redondeada, de hasta cinco centímetros de diámetro, negruzca por fuera y blanquecina o parda rojiza por dentro, y de muy buen olor. Se cría bajo tierra, y guisada es muy sabrosa. En otras zonas de Canarias se conoce con el nombre de criada.</>
<p><b>15</b> // <b>papa de color</b>. <i>Papa</i> de semilla autóctona, que entró en Canarias en el siglo XVI, la cual es de tamaño más bien pequeño, de pulpa más seca, compacta y azucarada, con muchos hoyitos u ojos, y cáscara de diversos colores, 
aunque predominan las gamas amarillosa, colorada y morada, propia de las
 medianías de la isla de Tenerife. Es muy apreciada en la cocina 
tradicional, y se consume principalmente arrugada.</>
<p><b>16</b> // <b>papa melonera</b>. <i>Papa borralla</i>.</>
<p><b>17</b> // <b>papa negra</b>. Variedad de <i>papa de color</i>, cuya cáscara es negra y la pulpa muy amarilla.</>
<p><b>18</b> // <b>papa palmera</b>. Variedad de <i>papa</i> parecida a la <i>bonita</i>, pero más alargada.</>
<p><b>19</b> // <b>papa quinegua</b>. Variedad de <i>papa</i> de importación, de forma alargada, de color blanco amarilloso y con el ojo rosado. Se caracteriza por producir mucho fruto, aunque de tamaño más pequeño que otras <i>papas</i>, como la <i>autodate</i> y la <i>cara</i>. <i>Las papas quineguas se emplean sobre todo para sancochar</i>.</>
<p><b>20</b> // <b>papa rosada</b>. Variedad de <i>papa</i> redonda y de cáscara de color blanco rosáceo. Tiene algo hundidos los ojos y, aunque se emplea sobre todo para guisar, es también apropiada para freír.</>
<p><b>21</b> // <b>papa terrenta</b>. <i>Papa borralla</i>.</>
<p><b>22</b> // <b>gustarle</b> a alguien <b>la papita suave/ dulce</b>.
 Tener una persona inclinación a trabajar lo menos posible y a llevar 
una vida cómoda. <i>Siempre le gustó la papita suave, pero ya le llegó la 
hora de tener que apencar</i>.</>
<p><b>23</b> // <b>no tener papas en la boca</b>. Decir sin reparo ni empacho lo que se piensa o lo que se siente. <i>Si tengo que decirle una verdad a alguien, yo no tengo papas en la boca</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Voz que, empleada sin determinante y en forma no apelativa, usa el hablante para referirse a su propio padre cuando habla con personas que no son sus hermanos. <i>Papá estuvo con tu padre en el cuartel</i>.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/papero/">papero</a> (<a href="http://www.academiacanarialengua.org/api/word/5119/">5119</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/papero/">papero</a> (<a href="http://www.academiacanarialengua.org/api/word/3594/">3594</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> (<i>Gallotia galloti palmae</i>) Macho adulto del lagarto común de la isla de La Palma, caracterizado por tener la cabeza bastante ennegrecida y la papada azul. Es más robusto que la hembra. Se conoce también con los nombres de <i>barbazul</i>, <i>barbol</i>, <i>barbolete</i> y <i>papazul</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Se dice de la persona que come muchas papas.</>
<p><b>2</b> Repartidor de papas. <i>Era un papero grande: se dedicaba a importar y exportar papas en gran cantidad</i>.</>
<p><b>3</b> En la siembra de la papa, persona encargada de echarla en el surco. <i>Y la papera viene atrás echando las papas al surco</i>.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/penco/">penco</a> (<a href="http://www.academiacanarialengua.org/api/word/3693/">3693</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/penco/">penco</a> (<a href="http://www.academiacanarialengua.org/api/word/3694/">3694</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Penca, planta de la familia de las cactáceas.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Persona despreciable. <i>Me parece que penco fue la palabra más bonita que le dijo</i>.</>
<p><b>2</b> Mujer de mala reputación, prostituta.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/pipano/">pipano</a> (<a href="http://www.academiacanarialengua.org/api/word/3817/">3817</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/pípano/">pípano</a> (<a href="http://www.academiacanarialengua.org/api/word/5142/">5142</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Pípano.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Dicho de una oveja, blanca con manchas canelas en las quijadas, las orejas y la cabeza.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/pita/">pita</a> (<a href="http://www.academiacanarialengua.org/api/word/3837/">3837</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/pita/">pita</a> (<a href="http://www.academiacanarialengua.org/api/word/3838/">3838</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Bocina de sonido potente que llevaban los vehículos.</>
<p><b>2</b> Bocina eléctrica de los vehículos. <i>Cada vez que hay un atasco, se ponen a tocar la pita</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> // <b>estallar(se) como una pita.</b> Sufrir un percance grave.</>
<p><b>2</b> Fracasar estrepitosamente.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/plaguear/">plaguear</a> (<a href="http://www.academiacanarialengua.org/api/word/3851/">3851</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/plaguear/">plaguear</a> (<a href="http://www.academiacanarialengua.org/api/word/3852/">3852</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Maldecir.</>
<p><b>2</b> Blasfemar.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Gemir y llorar, sollozando o clamando. <i>Estuvieron plagueando toda la noche al muerto</i>.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/puipano/">puipano</a> (<a href="http://www.academiacanarialengua.org/api/word/3934/">3934</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/puípano/">puípano</a> (<a href="http://www.academiacanarialengua.org/api/word/5143/">5143</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Se dice de la cabra blanca con manchas canelas o bermejas.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Pípano.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/rasca/">rasca</a> (<a href="http://www.academiacanarialengua.org/api/word/4033/">4033</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/rasca/">rasca</a> (<a href="http://www.academiacanarialengua.org/api/word/4034/">4034</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Borrachera. <i>En la despedida de soltero, todos se cogieron una rasca</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Rascapolvo.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/sable/">sable</a> (<a href="http://www.academiacanarialengua.org/api/word/5091/">5091</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/sable/">sable</a> (<a href="http://www.academiacanarialengua.org/api/word/5092/">5092</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Arena volcánica.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> (<i>Lepidopus caudatus</i>) Pez muy delgado, de cerca de 2 metros de largo y apenas 2 kilos de peso. Su piel es de color plateado y carece de escamas. Vive en aguas abiertas y profundas, principalmente entre los 250 y los 400 metros de profundidad. <i>El sable no es demasiado abundante en las costas de las Islas</i>.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/salón/">salón</a> (<a href="http://www.academiacanarialengua.org/api/word/4320/">4320</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/salón/">salón</a> (<a href="http://www.academiacanarialengua.org/api/word/4321/">4321</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Clase de tierra de color rojizo más o menos arcillosa.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Local amplio con puerta grande al exterior, que ocupa toda o parte de la planta baja de la vivienda principalmente rural, y que se suele utilizar como garaje, almacén o local comercial. <i>En la época de las fiestas, sacaba los coches del salón y allí montaba una especie de ventorrillo</i>.</>
<p><b>2</b> Dependencia de la gallera donde se ponen los gallos seleccionados para pelear.</>
<p><b>3</b> Lugar donde se preparan y se entrenan los gallos de pelea.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/samago/">samago</a> (<a href="http://www.academiacanarialengua.org/api/word/4332/">4332</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/sámago/">sámago</a> (<a href="http://www.academiacanarialengua.org/api/word/5140/">5140</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Joven o adolescente muy desarrollado, de conducta indolente.</>
<p><b>2</b> Animal joven muy desarrollado.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Escobajo del racimo.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/sierra/">sierra</a> (<a href="http://www.academiacanarialengua.org/api/word/4426/">4426</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/sierra/">sierra</a> (<a href="http://www.academiacanarialengua.org/api/word/4427/">4427</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Montón alargado de trigo y de otros frutos, como papas o peras. <i>Las mujeres quitaban con el baleo las impurezas que encontraban en las sierras de grano</i>.</>
<p><b>2</b> Montón de mies alargado en la era, que se hace antes de formar la parva.</>
<p><b>3</b> Conjunto de mollos que se disponen en dos tongas paralelas a lo largo del terreno, con las espigas hacia adentro.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> (<i>Sarda sarda</i>) Pez de la familia de los escómbridos, parecido a un atún pequeño, que no supera el metro de longitud. Es de color azulado con listas oscuras a lo largo del dorso y plateado en el vientre. <i>La sierra vive en aguas litorales y con frecuencia se acerca a las costas</i>.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/tacha/">tacha</a> (<a href="http://www.academiacanarialengua.org/api/word/4527/">4527</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/tacha/">tacha</a> (<a href="http://www.academiacanarialengua.org/api/word/4528/">4528</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Tacho.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Pieza metálica larga y delgada, con cabeza y punta, que sirve para asegurar una cosa a otra.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/tacho/">tacho</a> (<a href="http://www.academiacanarialengua.org/api/word/4529/">4529</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/tacho/">tacho</a> (<a href="http://www.academiacanarialengua.org/api/word/5110/">5110</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> En la fabricación de azúcar, aparato donde se evaporaba en vacío el jarabe hasta obtener una masa cristalizada.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Dicho de una cosa, que tiene menos elevación de lo normal.</>
<p><b>2</b> Tipo de caldero más ancho que alto.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/tajame/">tajame</a> (<a href="http://www.academiacanarialengua.org/api/word/4554/">4554</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/tájame/">tájame</a> (<a href="http://www.academiacanarialengua.org/api/word/5133/">5133</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> (<i>Rutheopsis herbanica</i>) Especie herbácea de la familia de las umbelíferas, perenne, de hojas pinnadas y flores blancas. Es un endemismo de las islas de Lanzarote y Fuerteventura.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Tajame.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/tajora/">tajora</a> (<a href="http://www.academiacanarialengua.org/api/word/4566/">4566</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/tajora/">tajora</a> (<a href="http://www.academiacanarialengua.org/api/word/4567/">4567</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Nombre que se da a diversos arbustos endémicos de las Islas, del género <i>Sideritis</i> y de la familia de las labiadas. Son semejantes a la salvia, y se caracterizan por tener recubiertas sus hojas por una fina y suave capa algodonosa. Se les atribuyen propiedades medicinales. En otras zonas de Canarias se conocen con los nombres de chajora, chajorra y chaora.</>
<p><b>2</b> (<i>Sideritis gomerae</i>) Planta de la familia de las labiadas, de hojas entre lanceoladas y oblongas de color blanco algodonoso, con inflorescencias colgantes formando una larga espiga. Es un endemismo de la isla de La Gomera, que se da en zonas rocosas.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Se dice de la cabra cuyo pelo tiene color blanco algodonoso, parecido al de la planta de este nombre.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/tanganillo/">tanganillo</a> (<a href="http://www.academiacanarialengua.org/api/word/4603/">4603</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/tanganillo/">tanganillo</a> (<a href="http://www.academiacanarialengua.org/api/word/4604/">4604</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Palo como de medio metro de largo, que se pone en el collar de los perros de caza en época de veda, para impedirles cazar.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Composición musical propia de las islas de Tenerife y La Palma, de ritmo alegre, que tradicionalmente se suele interpretar junto con el santodomingo y el tajaraste.</>
<p><b>2</b> Copla formada generalmente por versos de siete y cinco sílabas, de carácter festivo, que se interpreta al son de esta música.</>
<p><b>3</b> Baile que se ejecuta con esta música, en el que los danzantes cambian de pareja a medida que van girando en círculos.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/taro/">taro</a> (<a href="http://www.academiacanarialengua.org/api/word/5134/">5134</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/taró/">taró</a> (<a href="http://www.academiacanarialengua.org/api/word/4630/">4630</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Construcción rural en forma de torre circular, que se emplea con distintos fines, como, por ejemplo, curar el queso, guardar aperos o ser refugio de pastores. <i>El queso se mantenía más fresco en el taro que en los pajeros</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Vapor que, con la frialdad de la noche, se condensa en gotas muy menudas y que luego aparece sobre las plantas o sobre cualquier objeto que esté a la intemperie. <i>Cuando el tiempo está de levante, no hay taró</i>.</>
<p><b>2</b> Frío de la noche y del crepúsculo muy húmedo e intenso.</>
<p><b>3</b> Lluvia muy menuda.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/tesegue/">tesegue</a> (<a href="http://www.academiacanarialengua.org/api/word/5103/">5103</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/tesegue/">tesegue</a> (<a href="http://www.academiacanarialengua.org/api/word/4701/">4701</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Persona, animal o cosa de gran tamaño. <i>Nadie se atrevía a enfrentarse con aquel tesegue</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Tierra muy endurecida.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/teso/">teso</a> (<a href="http://www.academiacanarialengua.org/api/word/4703/">4703</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/teso/">teso</a> (<a href="http://www.academiacanarialengua.org/api/word/5104/">5104</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Fondo marino de suelo duro.</>
<p><b>2</b> Terreno de suelo muy duro. <i>En esos tesos se pasan las ovejas días enteros comiendo hierba</i>.</>
<p><b>3</b> Terreno que por la composición de su suelo es árido, improductivo.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Borracho, ebrio. <i>No le gusta verlo teso porque se pone muy pesado</i>.</>
<p><b>2</b> Entre los marineros, se dice del mar y de la calma cuando la quietud es total.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/titi/">titi</a> (<a href="http://www.academiacanarialengua.org/api/word/4750/">4750</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/tití/">tití</a> (<a href="http://www.academiacanarialengua.org/api/word/5141/">5141</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Tití.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Tratamiento familiar con que alguien se refiere a su <i>tío</i> o a su <i>tía</i>. <i>Todos queríamos que tití Carmela fuera la madrina</i>.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/tolda/">tolda</a> (<a href="http://www.academiacanarialengua.org/api/word/4764/">4764</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/tolda/">tolda</a> (<a href="http://www.academiacanarialengua.org/api/word/4765/">4765</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Agujero por donde se echa el grano en el molino de mano.</>
<p><b>2</b> Base de madera donde se coloca la solera o piedra inferior del molino de mano.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> (<i>Euphorbia aphylla</i>) Especie de tabaiba sin hojas, que crece fundamentalmente en ambientes costeros del norte de Gran Canaria y noroeste de Tenerife y de La Gomera. <i>Hoy es frecuente ver la tolda en jardines públicos y privados</i>.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/totizo/">totizo</a> (<a href="http://www.academiacanarialengua.org/api/word/4806/">4806</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/totizo/">totizo</a> (<a href="http://www.academiacanarialengua.org/api/word/5121/">5121</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Nuca, parte posterior de la cabeza. <i>Se dio un golpe en el totizo, y se quedó medio mareado</i>.</>
<p><b>2</b> Cuello muy grueso de una persona. <i>Aquel luchador tenía un totizo que llamaba la atención</i>.</>
<p><b>3</b> Cabeza. <i>Si apartara un poco el totizo, podríamos ver mejor</i>.</>
<p><b>4</b> Entendimiento. <i>No se le pasó por el totizo que pudieran perder</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Persona tarda en comprender. U. t. c. adj.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/totufo/">totufo</a> (<a href="http://www.academiacanarialengua.org/api/word/5122/">5122</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/totufo/">totufo</a> (<a href="http://www.academiacanarialengua.org/api/word/4809/">4809</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Abultamiento que se forma en la piel a consecuencia de un golpe u otra causa. <i>El golpe fue tan fuerte, que le salió un totufo en la cabeza</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Persona tarda en comprender. U. t. c. adj.&nbsp;<i>Aunque se creía una eminencia, en realidad era un totufo</i>.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/trinque/">trinque</a> (<a href="http://www.academiacanarialengua.org/api/word/4869/">4869</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/trinque/">trinque</a> (<a href="http://www.academiacanarialengua.org/api/word/4870/">4870</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> // <b>al/del trinque</b>. Se dice de las cosas que no han sido estrenadas. <i>Llevó la chaqueta a la tintorería y se la dieron que parecía del trinque</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Bebida alcohólica. Se usa sobre todo en expresiones como <i>gustarle a alguien el trinque</i>. <i>No le gusta el trinque, pero por las fiestas se suele tomar una copita de más</i>.</>
<p><b>2</b> Trago de bebida alcohólica.</>
</td>
</tr>
<tr>
<th> <a href="http://www.academiacanarialengua.org/palabra/vaqueta/">vaqueta</a> (<a href="http://www.academiacanarialengua.org/api/word/4952/">4952</a>)</th>
<th> <a href="http://www.academiacanarialengua.org/palabra/vaqueta/">vaqueta</a> (<a href="http://www.academiacanarialengua.org/api/word/4953/">4953</a>)</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Vaca, pez parecido a la cabrilla.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Persona gandula, haragana.</>
<p><b>2</b> Persona informal, trapacera.</>
<p><b>3</b> Persona cobarde.</>
</td>
</tr>
</table>

%%HTML

<style>

th {

    text-align: left;

    weight: bolder;

    background-color: #E9E9E9;

    color: black;

}

</style>

<table>

<tr>

<th> <a href="http://www.academiacanarialengua.org/palabra/andoriña/">andoriña</a> (<a href="http://www.academiacanarialengua.org/api/word/679/">679</a>)</th>

<th> <a href="http://www.academiacanarialengua.org/palabra/andoriña/">andoriña</a> (<a href="http://www.academiacanarialengua.org/api/word/5094/">5094</a>)</th>

</tr>

<tr>

<td style="vertical-align: top;">

<p><b>1</b> (<i>Apus unicolor</i>, <i>Apus pallidus brehmorum</i>, <i>Apus apus</i>) Nombre con el que se designa indistintamente a las tres especies de vencejos que nidifican en Canarias: vencejo unicolor, vencejo pálido y vencejo común. Son aves insectívoras que se alimentan en pleno vuelo y emiten unos característicos chillidos. Parcialmente migratorias, estas aves de unos quince centímetros son de color pardo negruzco. <i>Todavía me acuerdo de que a mi pueblo venían las andoriñas en bandadas volando bajito y siempre cazando</i>. En algunas zonas de Canarias se conocen también con los nombres de aburrión y golondrina.</>

<p><b>2</b> Persona inquieta y vivaz. <i>¿Y quién de nosotros es capaz de seguir el ritmo de esa andoriña</i><i>?</i></>

</td>

<td style="vertical-align: top;">

<p><b>1</b> Dicho de una cabra, totalmente negra. <i>Hay pastores que dicen que las cabras moriscas son más lecheras que las andoriñas</i>.</>

</td>

</tr>

</table>



andoriña (679),andoriña (5094)
"1 (Apus unicolor, Apus pallidus brehmorum, Apus apus) Nombre con el que se designa indistintamente a las tres especies de vencejos que nidifican en Canarias: vencejo unicolor, vencejo pálido y vencejo común. Son aves insectívoras que se alimentan en pleno vuelo y emiten unos característicos chillidos. Parcialmente migratorias, estas aves de unos quince centímetros son de color pardo negruzco. Todavía me acuerdo de que a mi pueblo venían las andoriñas en bandadas volando bajito y siempre cazando. En algunas zonas de Canarias se conocen también con los nombres de aburrión y golondrina.</> 2 Persona inquieta y vivaz. ¿Y quién de nosotros es capaz de seguir el ritmo de esa andoriña?</>","1 Dicho de una cabra, totalmente negra. Hay pastores que dicen que las cabras moriscas son más lecheras que las andoriñas.</>"
bago (858),bago (5129)
"1 Cada uno de los frutos que con otros iguales forman un racimo, como el de la uva o el plátano. Cogió un baguito de uva y se lo echó a la boca.</>","1 Dicho de ciertos frutos, especialmente de la aceituna, que tienen muy poca pulpa porque no alcanzan su desarrollo normal. U.?t. c. s. Mi madre, antes de ponerlas en su mojo, separaba las aceitunas bagas de las buenas.</> 2 Fruto de ciertas plantas, como el barbusano, el laurel o la pitera, que contiene una sola semilla, relativamente grande, recubierta de una capa carnosa delgada. De la baga de loro se llegó a hacer aceite.</>"
baifa (860),baifa (5112)
"1 Baifo hembra. Vendió una baifita en treinta euros.</> 2 Cabra de menos de un año, que puede ser cubierta.</> 3 Modorra, soñolencia. Después de comer, me entra una baifa, que no me deja hacer nada.</> 4 // estar (loco) como una baifa. Estar loco, chiflado.</>",1 Se dice de la papa fallida.</>
barbusano (905),barbusano (5096)
"1 (Apollonias barbujana) Árbol de la familia de las lauráceas, propio de las Islas Canarias y Madeira. Es de alto porte y su madera, extremadamente dura y resistente, es algo parecida a la caoba. Los árboles más altos eran los tiles, los mocanes y los barbusanos.</> 2 Madera de este árbol. Los ebanistas llaman al barbusano la caoba canaria.</>","1 Dicho de una cabra, blanca con el hocico rayado de negro.</>"
belete (957),beleté (5144)
"1 Beletén, primera leche que da la hembra después de parida.</> 2 Leche que dan las hembras los dos primeros días después del parto.</>","1 Beletén, primera leche que da la hembra después de parida.</>"
bicácaro (987),bicácaro (5097)


%%HTML
<style>
th {
    text-align: left;
    weight: bolder;
    background-color: #304060;
    color: white;
}
</style>

<table>
<tr><th colspan="2">andoriña</th></tr>
<tr>
<th>679</th>
<th>5094</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> (<i>Apus unicolor</i>, <i>Apus pallidus brehmorum</i>, <i>Apus apus</i>) Nombre con el que se designa indistintamente a las tres especies de vencejos que nidifican en Canarias: vencejo unicolor, vencejo pálido y vencejo común. Son aves insectívoras que se alimentan en pleno vuelo y emiten unos característicos chillidos. Parcialmente migratorias, estas aves de unos quince centímetros son de color pardo negruzco. <i>Todavía me acuerdo de que a mi pueblo venían las andoriñas en bandadas volando bajito y siempre cazando</i>. En algunas zonas de Canarias se conocen también con los nombres de aburrión y golondrina.</>
<p><b>2</b> Persona inquieta y vivaz. <i>¿Y quién de nosotros es capaz de seguir el ritmo de esa andoriña</i><i>?</i></>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Dicho de una cabra, totalmente negra. <i>Hay pastores que dicen que las cabras moriscas son más lecheras que las andoriñas</i>.</>
</td>
</tr>
<tr><th colspan="2">bago</th></tr>
<tr>
<th>858</th>
<th>5129</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Cada uno de los frutos que con otros iguales forman un racimo, como el de la uva o el plátano. <i>Cogió un baguito de uva y se lo echó a la boca.</i></>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Dicho de ciertos frutos, especialmente de la aceituna, que tienen muy poca pulpa porque no alcanzan su desarrollo normal. U.?t. c. s. <i>Mi madre, antes de ponerlas en su mojo, separaba las aceitunas bagas de las buenas</i>.</>
<p><b>2</b> Fruto de ciertas plantas, como el barbusano, el laurel o la pitera, que contiene una sola semilla, relativamente grande, recubierta de una capa carnosa delgada. <i>De la baga de loro se llegó a hacer aceite</i>.</>
</td>
</tr>
<tr><th colspan="2">baifa</th></tr>
<tr>
<th>860</th>
<th>5112</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> <i>Baifo</i> hembra. <i>Vendió una baifita en treinta euros</i>.</>
<p><b>2</b> Cabra de menos de un año, que puede ser cubierta.</>
<p><b>3</b> Modorra, soñolencia. <i>Después de comer, me entra una baifa, que no me deja hacer nada</i>.</>
<p><b>4</b> // <b>estar (loco) como una baifa</b>. Estar loco, chiflado.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Se dice de la papa fallida.</>
</td>
</tr>
<tr><th colspan="2">barbusano</th></tr>
<tr>
<th>905</th>
<th>5096</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> (<i>Apollonias barbujana</i>) Árbol de la familia de las lauráceas, propio de las Islas Canarias y Madeira. Es de alto porte y su madera, extremadamente dura y resistente, es algo parecida a la caoba. <i>Los árboles más altos eran los tiles, los mocanes y los barbusanos</i>.</>
<p><b>2</b> Madera de este árbol. <i>Los ebanistas llaman al barbusano la caoba canaria</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Dicho de una cabra, blanca con el hocico rayado de negro.</>
</td>
</tr>
<tr><th colspan="2">belete</th></tr>
<tr>
<th>957</th>
<th>5144</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> <i>Beletén</i>, primera leche que da la hembra después de parida.</>
<p><b>2</b> Leche que dan las hembras los dos primeros días después del parto.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> <i>Beletén</i>, primera leche que da la hembra después de parida.</>
</td>
</tr>
<tr><th colspan="2">bicácaro</th></tr>
<tr>
<th>987</th>
<th>5097</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Fruto de la <i>bicacarera</i>. <i>Los guanches se comían los bicácaros</i>.</>
<p><b>2</b> <i>Bicacarera</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Se dice del ojo y de la mirada torcida. <i>Este niño tiene un ojo medio bicácaro</i>.</>
<p><b>2</b> Que tiene el ojo o la mirada torcida.</>
</td>
</tr>
<tr><th colspan="2">borrallo</th></tr>
<tr>
<th>5113</th>
<th>5098</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Dicho de una cabra, de color ceniciento.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Embarcación sin cubierta, más pequeña que el barco de pozo, para pescar en el litoral. <i>En un descuido vino una ola y les reviró el bote</i>.</>
<p><b>2</b> Pequeña embarcación auxiliar con proa en ángulo y popa chata. <i>Salieron a pescar en la falúa, y dejaron el bote fondeado</i>.</>
<p><b>3</b> Embarcación que se utiliza en el deporte de la vela latina, que puede alcanzar hasta los seis metros y medio de eslora. <i>Todos los domingos se iba a la avenida a ver la regata de botes</i>.</>
</td>
</tr>
<tr><th colspan="2">bote</th></tr>
<tr>
<th>1086</th>
<th>5117</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> (<i>Delphinus delphis</i>) Especie de delfín, mamífero cetáceo.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Dicho de una persona, delgada. <i>A pesar de todo lo que comía, era muy cañizo</i>.</>
</td>
</tr>
<tr><th colspan="2">carajaca</th></tr>
<tr>
<th>5106</th>
<th>5107</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Plato compuesto por hígado de res vacuna cortado fino y troceado, frito y aderezado con salsa algo picante. <i>La especialidad de este bar son las carajacas y las vueltas de carne</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> (<i>Maja brachydactyla</i>) Crustáceo marino, variedad grande de cangrejo, que posee una concha muy dura y escabrosa con muchas púas y cinco pares de patas muy largas. <i>Aquella bahía tenía un fondo buenísimo: en una mañana se podía coger hasta una docena de carajacas</i>. En otras zonas de Canarias se conoce con el nombre de <i>santorra</i>.</>
</td>
</tr>
<tr><th colspan="2">carisco</th></tr>
<tr>
<th>1332</th>
<th>5136</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> (<i>Persea indica</i>) Árbol de la familia de las lauráceas, de hasta veinte metros de altura, con hojas lanceoladas de color verde pálido, que se vuelven rojizas al envejecer. Su fruto, de unos dos centímetros, tiene forma de aceituna y, al madurar, es de color negro azulado. Es frecuente en los bosques de <i>laurisilva</i>. En otras zonas de Canarias se conoce con los nombres de <i>viñático</i> y <i>viñátigo</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Carisco.</>
</td>
</tr>
<tr><th colspan="2">chopa</th></tr>
<tr>
<th>1536</th>
<th>1537</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Cucaracha, cuando es de tamaño grande.</>
<p><b>2</b> (<i>Rhyparobia maderae</i>) Cucaracha grande de color crema oscuro, que suele emprender vuelo en días de calor. En otras zonas de Canarias se conoce con los nombres de <i>cuca volona</i> y <i>salema</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Nariz, especialmente la abultada. <i>Con el catarro, tenía la chopa roja de tanto sonarse</i>.</>
</td>
</tr>
<tr><th colspan="2">chucho</th></tr>
<tr>
<th>1554</th>
<th>1555</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Látigo.</>
<p><b>2</b> Castigo físico. Se emplea sobre todo en expresiones como <i>dar chucho</i>, <i>alcanzar chucho</i>, etc. <i>Siempre lo amenazaba con darle chucho, pero nunca le llegó a pegar</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Nombre que se da a varias especies de peces con cuerpo en forma de raya, de cabeza puntiaguda o redondeada, cuerpo aplastado, liso y de color pardo o negruzco. Algunas de estas especies pueden alcanzar hasta los dos metros y medio de longitud y más de doscientos kilos de peso. Tienen una cola larga, que está provista de una aguda espina venenosa. Viven en aguas costeras, sobre fondos de arena y lodo, desde la orilla hasta los doscientos metros de profundidad. Aunque, por lo general, no suele consumirse fresca, su carne se aprovecha a veces para hacer <i>tollos</i>. <i>Los latigazos que daba el chucho con el rabo eran terribles</i>. En algunas zonas de Canarias se conocen también con el nombre de <i>chucha</i>.</>
<p><b>2</b> // <b>ser más feo que un chucho</b>. Ser muy poco agraciado.</>
</td>
</tr>
<tr><th colspan="2">corujo</th></tr>
<tr>
<th>1690</th>
<th>5151</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Fantasma con cuya mención se pretende amedrentar a los niños. <i>Nos decían que, si no nos portábamos bien, venía el corujo y nos llevaba</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Dicho de una cabra, de pelaje en que se combinan manchas negras y <i>canelas</i>.</>
</td>
</tr>
<tr><th colspan="2">cuca</th></tr>
<tr>
<th>1731</th>
<th>1732</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Cucaracha. <i>Este verano, como hizo mucho calor, se le llenó la casa de cucas</i>.</>
<p><b>2</b> Órgano sexual masculino, especialmente el de los niños.</>
<p><b>3</b> // <b>cuca inglesa</b>. (<i>Blatella germanica</i>) Cucaracha pequeña de color rubio. Se encuentra principalmente en las zonas costeras de clima cálido. En otras zonas de Canarias se conoce con el nombre de <i>chiripa</i>.</>
<p><b>4</b> // <b>cuca volona</b>. (<i>Rhyparobia maderae</i>) Cucaracha grande de color crema oscuro, que suele emprender vuelo en días de calor. En algunas zonas de Canarias se conoce también con los nombres de <i>chopa</i> y <i>salema</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Aplicado a cabras, de color negro.</>
</td>
</tr>
<tr><th colspan="2">empanado</th></tr>
<tr>
<th>5145</th>
<th>1960</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Conjunto de panas de una embarcación, dispuestas de proa a popa. <i>Hay que sacar todo el empanado para poder limpiar bien el barco por dentro</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Dicho de una plantación, que tiene sus elementos muy juntos o apretados.<i> Esta cebada creció bien empanadita.</i></>
</td>
</tr>
<tr><th colspan="2">fato</th></tr>
<tr>
<th>5131</th>
<th>5132</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Vaho, mal aliento.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Trapo, andrajo. <i>Se conoce que iba a pulpear, porque llevaba un fato y una fija en la mano.</i></>
</td>
</tr>
<tr><th colspan="2">funche</th></tr>
<tr>
<th>2443</th>
<th>2444</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Pequeño departamento dentro de las viviendas, comúnmente de reducidas dimensiones y desprovisto de luz y ventilación, que sirve de trastero.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Millo sin tostar molido muy fino.</>
<p><b>2</b> Vianda que se prepara con este millo molido. Por lo general, se come en frío añadiéndole leche o tabefe. Modernamente se toma también como postre, agregándole otros ingredientes, como canela en rama, corteza de limón, pasas y almendras.</>
</td>
</tr>
<tr><th colspan="2">galletón</th></tr>
<tr>
<th>2476</th>
<th>2477</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Adolescente, muchacho. <i>Aunque era un galletón, todavía iba a buscar nidos de pájaros</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Galleta grande. <i>Para desayunar le daban un galletón y un vaso de leche</i>.</>
</td>
</tr>
<tr><th colspan="2">gavia</th></tr>
<tr>
<th>2548</th>
<th>2549</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Parcela de terreno para cultivar, bordeada por un caballón de tierra, que se riega por encharcamiento con el agua de lluvia procedente de la alcogida. <i>Todo eso eran gavias, y ahora está todo lleno de casas</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Parte más alta de un árbol.</>
</td>
</tr>
<tr><th colspan="2">guanil</th></tr>
<tr>
<th>2651</th>
<th>5146</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Dicho del ganado que se cría suelto, que no tiene marca que identifique al dueño. <i>En la apañada se marcaban las crías que estaban guaniles</i>.</>
<p><b>2</b> Dicho de persona, animal o cosa, que, teniendo pareja, se queda solo. <i>Ya no le queda más que una tarraya vieja y un trasmallo guanil</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Se dice de la persona de corta edad.</>
</td>
</tr>
<tr><th colspan="2">majada</th></tr>
<tr>
<th>3132</th>
<th>3133</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Cerca móvil y desmontable formada por lances, que se instala en el campo y que se usa para encerrar el rebaño de ovejas.</>
<p><b>2</b> Depresión que se forma en la ladera de una montaña.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Acción y efecto de majarse, machacarse o aplastarse algún miembro del cuerpo.</>
<p><b>2</b> Acción y efecto de majarse, aprisionarse con daño la piel o algún miembro del cuerpo.</>
<p><b>3</b> Serie de golpes dados a una persona que la dejan maltrecha, paliza.</>
</td>
</tr>
<tr><th colspan="2">mano</th></tr>
<tr>
<th>3203</th>
<th>3204</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Fórmula apelativa, aféresis de <i>hermano</i>, usada con el valor de compañero, que se emplea entre iguales y en situaciones de habla muy informales. <i>Mano, termina uno cansándose de tanto abuso</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Manilla.</>
<p><b>2</b> Cada uno de los racimos de fruto que echa el tomatero.</>
<p><b>3</b> Grupo de personas, comúnmente de baja calidad. <i>En esta plaza se junta todas las noches una mano de golfantes y gamberros</i>.</>
<p><b>4</b> En expresiones como <i>una mano de estampidos</i> o <i>una mano de estacazos</i>, gran cantidad o gran número. <i>Tanto hizo, hasta que alcanzó una mano de cogotazos</i>.</>
</td>
</tr>
<tr><th colspan="2">margullo</th></tr>
<tr>
<th>3240</th>
<th>3241</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Acción de margullar o margullir.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Marullo.</>
</td>
</tr>
<tr><th colspan="2">mimo</th></tr>
<tr>
<th>3346</th>
<th>3347</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> (<i>Nicotiana glauca</i>) Arbusto de la familia de las solanáceas, con hojas lanceoladas y flores amarillas de forma tubular, muy abundante en lugares donde se vierten escombros y en los bordes de la carretera. En otras zonas de Canarias se conoce con los nombres de <i>bobo</i> y <i>tabaco moro</i>.</>
<p><b>2</b> (<i>Fuchsia magellanica</i>) Pequeña planta arbustiva de jardín, de la familia de las oenoteráceas, con flores de color rojizo en forma de pequeña campanita.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Dulce de forma redondeada y cocido al horno, hecho con clara de huevo, azúcar y raspas de limón. <i>En unos sitios le dicen mimo y en otros, merengue</i>.</>
</td>
</tr>
<tr><th colspan="2">mole</th></tr>
<tr>
<th>3370</th>
<th>3371</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Mol.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Blando. <i>Los plátanos, si se pasan de maduros, se ponen moles</i>.</>
</td>
</tr>
<tr><th colspan="2">mujo</th></tr>
<tr>
<th>3443</th>
<th>5149</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Musgo, planta.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Musgo.</>
</td>
</tr>
<tr><th colspan="2">musgo</th></tr>
<tr>
<th>5150</th>
<th>3455</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Nombre que se da, además de a las plantas muscíneas, a las algas y a los líquenes.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Se dice de la cabra que tiene las orejas más pequeñas que las de tamaño normal.</>
<p><b>2</b> Se dice de la cabra que tiene las orejas más grandes que la muesa y más pequeñas que las de tamaño normal.</>
</td>
</tr>
<tr><th colspan="2">ñoño</th></tr>
<tr>
<th>5086</th>
<th>5124</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Dedo del pie. U. m. en pl.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Se dice de la persona que tiene la mandíbula inferior más saliente que la superior. <i>Yo soy ñoña, pero mi hija no salió a mí en eso</i>.</>
</td>
</tr>
<tr><th colspan="2">papa</th></tr>
<tr>
<th>3582</th>
<th>5123</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> (<i>Solanum tuberosum</i>) Planta de la familia de las solanáceas, originaria de América. <i>El cultivo de la papa se introdujo en Canarias en el siglo XVI</i>.</>
<p><b>2</b> Tubérculo de dicha planta. <i>En la comida del canario no podían faltar las papas y el gofio</i>. (Fig. 109)</>
<p><b>3</b> Inflamación de las glándulas parótidas. <i>Tenía unas papas tremendas en la garganta, y no podía ni tragar</i>.</>
<p><b>4</b> Abultamiento cutáneo. <i>Le salió una papa en el pescuezo</i>.</>
<p><b>5</b> Tanto. <i>Vaya papa metió el delantero centro</i>.</>
<p><b>6</b> // <b>papa arrugada</b>. La que es guisada con cáscara y bastante sal.</>
<p><b>7</b> // <b>papa autodate</b>. Variedad de <i>papa</i>, alargada y de color blanco, muy estimada para comer. <i>Ya tú ves: la papa autodate es buenísima para comer, pero muy pudrilona</i>.</>
<p><b>8</b> // <b>papa azucena</b>. Clase de <i>papa de color</i>, de pulpa blanquecina, la cual presenta dos variedades distintas, que, de acuerdo con el color de la cáscara, se denominan <i>azucena blanca</i> y <i>azucena negra</i>. <i>Para acompañar el pescado, nos arrugaron unas papas azucenas, que estaban buenísimas</i>.</>
<p><b>9</b> // <b>papa blanca</b>. <i>Papa</i> de semilla foránea, importada generalmente de las Islas Británicas, la cual es de tamaño mediano o grande, de pulpa algo aguanosa, con pocos ojos, y cáscara de diversos colores, aunque predominan las gamas blanca, amarillosa y rosácea. Se emplea sobre todo para freír y para caldos y potajes, pero también para guisar, tanto pelada como con cáscara. <i>Hoy la papa blanca que más se está vendiendo es la quinegua, la rosada y la autodate</i>. <b>2</b>.Variedad de <i>papa</i>, redonda y de color blanco, con muchos ojos. Es una clase poco cultivada en la actualidad, debido a que es dura para comer.</>
<p><b>10</b> // <b>papa bonita</b>. Variedad de <i>papa de color</i>, de cáscara blanca amarillosa y pulpa compacta y azucarada. Es la más abundante y conocida de dicha clase de <i>papas</i>. <i>Fui a la recova a comprar papas bonitas, pero están por las nubes</i>. <b>2</b> <i>Tf.</i> <i>Papa de color</i>.</>
<p><b>11</b> // <b>papa borralla</b>. Variedad de <i>papa de 
color</i>, algo mayor que la <i>bonita</i> y de superficie más irregular, debido a 
que presenta muchos ojos y más hondos que los de esta. Su pulpa es algo más seca y compacta. <i>Las papas borrallas de la zona de Anaga dicen que son las mejores</i>.</>
<p><b>12</b> // <b>papa cara</b>. Variedad híbrida de <i>papa</i> de importación, que, aunque se considera que no es de la mejor calidad, se cultiva por su gran rendimiento.</>
<p><b>13</b> // <b>papa chinegua</b>. <i>Papa quinegua</i>.</>
<p><b>14</b> // <b>papa cría</b>. (<i>Terfezia claveryi</i>) Especie de trufa de forma redondeada, de hasta cinco centímetros de diámetro, negruzca por fuera y blanquecina o parda rojiza por dentro, y de muy buen olor. Se cría bajo tierra, y guisada es muy sabrosa. En otras zonas de Canarias se conoce con el nombre de criada.</>
<p><b>15</b> // <b>papa de color</b>. <i>Papa</i> de semilla autóctona, que entró en Canarias en el siglo XVI, la cual es de tamaño más bien pequeño, de pulpa más seca, compacta y azucarada, con muchos hoyitos u ojos, y cáscara de diversos colores, 
aunque predominan las gamas amarillosa, colorada y morada, propia de las
 medianías de la isla de Tenerife. Es muy apreciada en la cocina 
tradicional, y se consume principalmente arrugada.</>
<p><b>16</b> // <b>papa melonera</b>. <i>Papa borralla</i>.</>
<p><b>17</b> // <b>papa negra</b>. Variedad de <i>papa de color</i>, cuya cáscara es negra y la pulpa muy amarilla.</>
<p><b>18</b> // <b>papa palmera</b>. Variedad de <i>papa</i> parecida a la <i>bonita</i>, pero más alargada.</>
<p><b>19</b> // <b>papa quinegua</b>. Variedad de <i>papa</i> de importación, de forma alargada, de color blanco amarilloso y con el ojo rosado. Se caracteriza por producir mucho fruto, aunque de tamaño más pequeño que otras <i>papas</i>, como la <i>autodate</i> y la <i>cara</i>. <i>Las papas quineguas se emplean sobre todo para sancochar</i>.</>
<p><b>20</b> // <b>papa rosada</b>. Variedad de <i>papa</i> redonda y de cáscara de color blanco rosáceo. Tiene algo hundidos los ojos y, aunque se emplea sobre todo para guisar, es también apropiada para freír.</>
<p><b>21</b> // <b>papa terrenta</b>. <i>Papa borralla</i>.</>
<p><b>22</b> // <b>gustarle</b> a alguien <b>la papita suave/ dulce</b>.
 Tener una persona inclinación a trabajar lo menos posible y a llevar 
una vida cómoda. <i>Siempre le gustó la papita suave, pero ya le llegó la 
hora de tener que apencar</i>.</>
<p><b>23</b> // <b>no tener papas en la boca</b>. Decir sin reparo ni empacho lo que se piensa o lo que se siente. <i>Si tengo que decirle una verdad a alguien, yo no tengo papas en la boca</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Voz que, empleada sin determinante y en forma no apelativa, usa el hablante para referirse a su propio padre cuando habla con personas que no son sus hermanos. <i>Papá estuvo con tu padre en el cuartel</i>.</>
</td>
</tr>
<tr><th colspan="2">papero</th></tr>
<tr>
<th>5119</th>
<th>3594</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> (<i>Gallotia galloti palmae</i>) Macho adulto del lagarto común de la isla de La Palma, caracterizado por tener la cabeza bastante ennegrecida y la papada azul. Es más robusto que la hembra. Se conoce también con los nombres de <i>barbazul</i>, <i>barbol</i>, <i>barbolete</i> y <i>papazul</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Se dice de la persona que come muchas papas.</>
<p><b>2</b> Repartidor de papas. <i>Era un papero grande: se dedicaba a importar y exportar papas en gran cantidad</i>.</>
<p><b>3</b> En la siembra de la papa, persona encargada de echarla en el surco. <i>Y la papera viene atrás echando las papas al surco</i>.</>
</td>
</tr>
<tr><th colspan="2">penco</th></tr>
<tr>
<th>3693</th>
<th>3694</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Penca, planta de la familia de las cactáceas.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Persona despreciable. <i>Me parece que penco fue la palabra más bonita que le dijo</i>.</>
<p><b>2</b> Mujer de mala reputación, prostituta.</>
</td>
</tr>
<tr><th colspan="2">pipano</th></tr>
<tr>
<th>3817</th>
<th>5142</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Pípano.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Dicho de una oveja, blanca con manchas canelas en las quijadas, las orejas y la cabeza.</>
</td>
</tr>
<tr><th colspan="2">pita</th></tr>
<tr>
<th>3837</th>
<th>3838</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Bocina de sonido potente que llevaban los vehículos.</>
<p><b>2</b> Bocina eléctrica de los vehículos. <i>Cada vez que hay un atasco, se ponen a tocar la pita</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> // <b>estallar(se) como una pita.</b> Sufrir un percance grave.</>
<p><b>2</b> Fracasar estrepitosamente.</>
</td>
</tr>
<tr><th colspan="2">plaguear</th></tr>
<tr>
<th>3851</th>
<th>3852</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Maldecir.</>
<p><b>2</b> Blasfemar.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Gemir y llorar, sollozando o clamando. <i>Estuvieron plagueando toda la noche al muerto</i>.</>
</td>
</tr>
<tr><th colspan="2">puipano</th></tr>
<tr>
<th>3934</th>
<th>5143</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Se dice de la cabra blanca con manchas canelas o bermejas.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Pípano.</>
</td>
</tr>
<tr><th colspan="2">rasca</th></tr>
<tr>
<th>4033</th>
<th>4034</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Borrachera. <i>En la despedida de soltero, todos se cogieron una rasca</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Rascapolvo.</>
</td>
</tr>
<tr><th colspan="2">sable</th></tr>
<tr>
<th>5091</th>
<th>5092</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Arena volcánica.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> (<i>Lepidopus caudatus</i>) Pez muy delgado, de cerca de 2 metros de largo y apenas 2 kilos de peso. Su piel es de color plateado y carece de escamas. Vive en aguas abiertas y profundas, principalmente entre los 250 y los 400 metros de profundidad. <i>El sable no es demasiado abundante en las costas de las Islas</i>.</>
</td>
</tr>
<tr><th colspan="2">salón</th></tr>
<tr>
<th>4320</th>
<th>4321</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Clase de tierra de color rojizo más o menos arcillosa.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Local amplio con puerta grande al exterior, que ocupa toda o parte de la planta baja de la vivienda principalmente rural, y que se suele utilizar como garaje, almacén o local comercial. <i>En la época de las fiestas, sacaba los coches del salón y allí montaba una especie de ventorrillo</i>.</>
<p><b>2</b> Dependencia de la gallera donde se ponen los gallos seleccionados para pelear.</>
<p><b>3</b> Lugar donde se preparan y se entrenan los gallos de pelea.</>
</td>
</tr>
<tr><th colspan="2">samago</th></tr>
<tr>
<th>4332</th>
<th>5140</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Joven o adolescente muy desarrollado, de conducta indolente.</>
<p><b>2</b> Animal joven muy desarrollado.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Escobajo del racimo.</>
</td>
</tr>
<tr><th colspan="2">sierra</th></tr>
<tr>
<th>4426</th>
<th>4427</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Montón alargado de trigo y de otros frutos, como papas o peras. <i>Las mujeres quitaban con el baleo las impurezas que encontraban en las sierras de grano</i>.</>
<p><b>2</b> Montón de mies alargado en la era, que se hace antes de formar la parva.</>
<p><b>3</b> Conjunto de mollos que se disponen en dos tongas paralelas a lo largo del terreno, con las espigas hacia adentro.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> (<i>Sarda sarda</i>) Pez de la familia de los escómbridos, parecido a un atún pequeño, que no supera el metro de longitud. Es de color azulado con listas oscuras a lo largo del dorso y plateado en el vientre. <i>La sierra vive en aguas litorales y con frecuencia se acerca a las costas</i>.</>
</td>
</tr>
<tr><th colspan="2">tacha</th></tr>
<tr>
<th>4527</th>
<th>4528</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Tacho.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Pieza metálica larga y delgada, con cabeza y punta, que sirve para asegurar una cosa a otra.</>
</td>
</tr>
<tr><th colspan="2">tacho</th></tr>
<tr>
<th>4529</th>
<th>5110</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> En la fabricación de azúcar, aparato donde se evaporaba en vacío el jarabe hasta obtener una masa cristalizada.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Dicho de una cosa, que tiene menos elevación de lo normal.</>
<p><b>2</b> Tipo de caldero más ancho que alto.</>
</td>
</tr>
<tr><th colspan="2">tajame</th></tr>
<tr>
<th>4554</th>
<th>5133</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> (<i>Rutheopsis herbanica</i>) Especie herbácea de la familia de las umbelíferas, perenne, de hojas pinnadas y flores blancas. Es un endemismo de las islas de Lanzarote y Fuerteventura.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Tajame.</>
</td>
</tr>
<tr><th colspan="2">tajora</th></tr>
<tr>
<th>4566</th>
<th>4567</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Nombre que se da a diversos arbustos endémicos de las Islas, del género <i>Sideritis</i> y de la familia de las labiadas. Son semejantes a la salvia, y se caracterizan por tener recubiertas sus hojas por una fina y suave capa algodonosa. Se les atribuyen propiedades medicinales. En otras zonas de Canarias se conocen con los nombres de chajora, chajorra y chaora.</>
<p><b>2</b> (<i>Sideritis gomerae</i>) Planta de la familia de las labiadas, de hojas entre lanceoladas y oblongas de color blanco algodonoso, con inflorescencias colgantes formando una larga espiga. Es un endemismo de la isla de La Gomera, que se da en zonas rocosas.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Se dice de la cabra cuyo pelo tiene color blanco algodonoso, parecido al de la planta de este nombre.</>
</td>
</tr>
<tr><th colspan="2">tanganillo</th></tr>
<tr>
<th>4603</th>
<th>4604</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Palo como de medio metro de largo, que se pone en el collar de los perros de caza en época de veda, para impedirles cazar.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Composición musical propia de las islas de Tenerife y La Palma, de ritmo alegre, que tradicionalmente se suele interpretar junto con el santodomingo y el tajaraste.</>
<p><b>2</b> Copla formada generalmente por versos de siete y cinco sílabas, de carácter festivo, que se interpreta al son de esta música.</>
<p><b>3</b> Baile que se ejecuta con esta música, en el que los danzantes cambian de pareja a medida que van girando en círculos.</>
</td>
</tr>
<tr><th colspan="2">taro</th></tr>
<tr>
<th>5134</th>
<th>4630</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Construcción rural en forma de torre circular, que se emplea con distintos fines, como, por ejemplo, curar el queso, guardar aperos o ser refugio de pastores. <i>El queso se mantenía más fresco en el taro que en los pajeros</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Vapor que, con la frialdad de la noche, se condensa en gotas muy menudas y que luego aparece sobre las plantas o sobre cualquier objeto que esté a la intemperie. <i>Cuando el tiempo está de levante, no hay taró</i>.</>
<p><b>2</b> Frío de la noche y del crepúsculo muy húmedo e intenso.</>
<p><b>3</b> Lluvia muy menuda.</>
</td>
</tr>
<tr><th colspan="2">tesegue</th></tr>
<tr>
<th>5103</th>
<th>4701</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Persona, animal o cosa de gran tamaño. <i>Nadie se atrevía a enfrentarse con aquel tesegue</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Tierra muy endurecida.</>
</td>
</tr>
<tr><th colspan="2">teso</th></tr>
<tr>
<th>4703</th>
<th>5104</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Fondo marino de suelo duro.</>
<p><b>2</b> Terreno de suelo muy duro. <i>En esos tesos se pasan las ovejas días enteros comiendo hierba</i>.</>
<p><b>3</b> Terreno que por la composición de su suelo es árido, improductivo.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Borracho, ebrio. <i>No le gusta verlo teso porque se pone muy pesado</i>.</>
<p><b>2</b> Entre los marineros, se dice del mar y de la calma cuando la quietud es total.</>
</td>
</tr>
<tr><th colspan="2">titi</th></tr>
<tr>
<th>4750</th>
<th>5141</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Tití.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Tratamiento familiar con que alguien se refiere a su <i>tío</i> o a su <i>tía</i>. <i>Todos queríamos que tití Carmela fuera la madrina</i>.</>
</td>
</tr>
<tr><th colspan="2">tolda</th></tr>
<tr>
<th>4764</th>
<th>4765</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Agujero por donde se echa el grano en el molino de mano.</>
<p><b>2</b> Base de madera donde se coloca la solera o piedra inferior del molino de mano.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> (<i>Euphorbia aphylla</i>) Especie de tabaiba sin hojas, que crece fundamentalmente en ambientes costeros del norte de Gran Canaria y noroeste de Tenerife y de La Gomera. <i>Hoy es frecuente ver la tolda en jardines públicos y privados</i>.</>
</td>
</tr>
<tr><th colspan="2">totizo</th></tr>
<tr>
<th>4806</th>
<th>5121</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Nuca, parte posterior de la cabeza. <i>Se dio un golpe en el totizo, y se quedó medio mareado</i>.</>
<p><b>2</b> Cuello muy grueso de una persona. <i>Aquel luchador tenía un totizo que llamaba la atención</i>.</>
<p><b>3</b> Cabeza. <i>Si apartara un poco el totizo, podríamos ver mejor</i>.</>
<p><b>4</b> Entendimiento. <i>No se le pasó por el totizo que pudieran perder</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Persona tarda en comprender. U. t. c. adj.</>
</td>
</tr>
<tr><th colspan="2">totufo</th></tr>
<tr>
<th>5122</th>
<th>4809</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Abultamiento que se forma en la piel a consecuencia de un golpe u otra causa. <i>El golpe fue tan fuerte, que le salió un totufo en la cabeza</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Persona tarda en comprender. U. t. c. adj.&nbsp;<i>Aunque se creía una eminencia, en realidad era un totufo</i>.</>
</td>
</tr>
<tr><th colspan="2">trinque</th></tr>
<tr>
<th>4869</th>
<th>4870</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> // <b>al/del trinque</b>. Se dice de las cosas que no han sido estrenadas. <i>Llevó la chaqueta a la tintorería y se la dieron que parecía del trinque</i>.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Bebida alcohólica. Se usa sobre todo en expresiones como <i>gustarle a alguien el trinque</i>. <i>No le gusta el trinque, pero por las fiestas se suele tomar una copita de más</i>.</>
<p><b>2</b> Trago de bebida alcohólica.</>
</td>
</tr>
<tr><th colspan="2">vaqueta</th></tr>
<tr>
<th>4952</th>
<th>4953</th>
</tr>
<tr>
<td style="vertical-align: top;">
<p><b>1</b> Vaca, pez parecido a la cabrilla.</>
</td>
<td style="vertical-align: top;">
<p><b>1</b> Persona gandula, haragana.</>
<p><b>2</b> Persona informal, trapacera.</>
<p><b>3</b> Persona cobarde.</>
</td>
</tr>
</table>